# StationSim_GCS plotly animation example

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import sys
from tqdm import tqdm

In [2]:
sys.path.append('../../../stationsim/')

from stationsim_gcs_model import Model

## Running the model

In [3]:
# Set parameter values
model_params = {'pop_total': 100,
                'station': 'Grand_Central',
                'do_print': False}

In [4]:
# Create model instance
model = Model(**model_params)

In [5]:
# Set model forward
while model.status:
    model.step()

## Creating an animation

When making the animation using plotly, it is a good idea to provide a `sampling frequency`.
This variable indicates the step interval between observations of the agent positions.
We do not require the model state at every timestep to be included, as plotly links each of the agents together, providing a smooth animation of each of the agents.

A `sampling_frequency` of 1 would indicate that we wish to include every timestep for each of the agents; a `sampling_frequency` of 5 would indicate that we wish to include every 5th timestep in the animation.

The larger the `sampling_frequency`, the fewer observations are included in the animation and consequently the less time required to produce it; however, larger the `sampling_frequency`, the greater the number of timesteps between observed states and consequently the lower the temporal resolution of the agent trails.

In [6]:
sampling_frequency = 5

Having a defined a `sampling_frequency`, we can create a dataframe of the agent coordinates from the model:

In [7]:
model_history = list()

for step, state in enumerate(model.history_state):
    if step % sampling_frequency == 0:
        for agent_id, agent_coords in enumerate(state):
            d = {'agent_id': agent_id,
                 't': step,
                 'x': agent_coords[0],
                 'y': agent_coords[1]}
            model_history.append(d)

model_history = pd.DataFrame(model_history)

In [8]:
len(model_history)

87000

In [9]:
model_history.head()

,agent_id,t,x,y
0,0,0,0.0,0.0
1,1,0,0.0,0.0
2,2,0,0.0,0.0
3,3,0,0.0,0.0
4,4,0,0.0,0.0


Finally, we can produce an animated scatter plot of the agent locations over the course of the model lifettime, showing how the agents move within the model environment.

In [10]:
# Produce animated scatter plot
f = px.scatter(model_history, x='x', y='y',
               animation_frame='t', animation_group='agent_id',
               range_x=[0, model.width], range_y=[0, model.height])

In [11]:
# Write animation to html
f.write_html('./stationsim_gcs_animation.html')

We can now open `./stationsim_gcs_animation.html` in a web browser to have a look at the animation.